In [1]:
import requests
from datetime import datetime, timezone, timedelta
import json
import pandas as pd
from trust_algorithm import trust_alg

In [2]:
url = 'http://127.0.0.1:8000/locationConsensus/interactions/'
# url = "https://dsc180-decentralized-location.herokuapp.com/locationConsensus/interactions/"
data = requests.get(url).json()
data

[{'from_user': 'alan',
  'spotted_users': '["nathan", "mason", "alex"]',
  'time': '2023-02-09T01:09:27Z'},
 {'from_user': 'mason',
  'spotted_users': '["alan"]',
  'time': '2023-02-09T01:09:27Z'},
 {'from_user': 'nathan',
  'spotted_users': '["alan"]',
  'time': '2023-02-09T01:09:27Z'},
 {'from_user': 'alex',
  'spotted_users': '["alan"]',
  'time': '2023-02-09T01:09:27Z'},
 {'from_user': 'alan',
  'spotted_users': '["nathan", "mason", "alex"]',
  'time': '2023-02-09T01:09:27Z'},
 {'from_user': 'mason',
  'spotted_users': '["alan"]',
  'time': '2023-02-09T01:09:27Z'},
 {'from_user': 'nathan',
  'spotted_users': '["alan"]',
  'time': '2023-02-09T01:09:27Z'},
 {'from_user': 'alex',
  'spotted_users': '["alan"]',
  'time': '2023-02-09T01:09:27Z'}]

In [3]:
df = pd.DataFrame(data)
df

,from_user,spotted_users,time
0,alan,"[""nathan"", ""mason"", ""alex""]",2023-02-09T01:09:27Z
1,mason,"[""alan""]",2023-02-09T01:09:27Z
2,nathan,"[""alan""]",2023-02-09T01:09:27Z
3,alex,"[""alan""]",2023-02-09T01:09:27Z
4,alan,"[""nathan"", ""mason"", ""alex""]",2023-02-09T01:09:27Z
5,mason,"[""alan""]",2023-02-09T01:09:27Z
6,nathan,"[""alan""]",2023-02-09T01:09:27Z
7,alex,"[""alan""]",2023-02-09T01:09:27Z


In [25]:
# df["time"] = df["time"].str.split(".").str[0]
df["time"] = pd.to_datetime(df["time"])
df

,from_user,spotted_users,time
0,alan,"[""nathan"", ""mason"", ""alex""]",2023-02-09 01:09:27+00:00
1,mason,"[""alan""]",2023-02-09 01:09:27+00:00
2,nathan,"[""alan""]",2023-02-09 01:09:27+00:00
3,alex,"[""alan""]",2023-02-09 01:09:27+00:00
4,alan,"[""nathan"", ""mason"", ""alex""]",2023-02-09 01:09:27+00:00
5,mason,"[""alan""]",2023-02-09 01:09:27+00:00
6,nathan,"[""alan""]",2023-02-09 01:09:27+00:00
7,alex,"[""alan""]",2023-02-09 01:09:27+00:00


In [5]:
now = datetime.now(timezone.utc)
print(now)
past_hour = now - timedelta(hours=1)
print(past_hour)

2023-02-16 01:14:40.701624+00:00
2023-02-16 00:14:40.701624+00:00


In [6]:
current_df = df.loc[df["time"] > past_hour]
current_df

,from_user,spotted_users,time


In [7]:
df

,from_user,spotted_users,time
0,alan,"[""nathan"", ""mason"", ""alex""]",2023-02-09 01:09:27+00:00
1,mason,"[""alan""]",2023-02-09 01:09:27+00:00
2,nathan,"[""alan""]",2023-02-09 01:09:27+00:00
3,alex,"[""alan""]",2023-02-09 01:09:27+00:00
4,alan,"[""nathan"", ""mason"", ""alex""]",2023-02-09 01:09:27+00:00
5,mason,"[""alan""]",2023-02-09 01:09:27+00:00
6,nathan,"[""alan""]",2023-02-09 01:09:27+00:00
7,alex,"[""alan""]",2023-02-09 01:09:27+00:00


In [26]:
df["spotted_users"] = df["spotted_users"].apply(json.loads)
df

,from_user,spotted_users,time
0,alan,"[nathan, mason, alex]",2023-02-09 01:09:27+00:00
1,mason,[alan],2023-02-09 01:09:27+00:00
2,nathan,[alan],2023-02-09 01:09:27+00:00
3,alex,[alan],2023-02-09 01:09:27+00:00
4,alan,"[nathan, mason, alex]",2023-02-09 01:09:27+00:00
5,mason,[alan],2023-02-09 01:09:27+00:00
6,nathan,[alan],2023-02-09 01:09:27+00:00
7,alex,[alan],2023-02-09 01:09:27+00:00


In [12]:
df = df.explode("spotted_users")
df

,from_user,spotted_users,time
0,alan,nathan,2023-02-09 01:09:27+00:00
0,alan,mason,2023-02-09 01:09:27+00:00
0,alan,alex,2023-02-09 01:09:27+00:00
1,mason,alan,2023-02-09 01:09:27+00:00
2,nathan,alan,2023-02-09 01:09:27+00:00
3,alex,alan,2023-02-09 01:09:27+00:00
4,alan,nathan,2023-02-09 01:09:27+00:00
4,alan,mason,2023-02-09 01:09:27+00:00
4,alan,alex,2023-02-09 01:09:27+00:00
5,mason,alan,2023-02-09 01:09:27+00:00


In [11]:
spotted_percentage = current_df.groupby("spotted_users")["from_user"].count() / current_df["from_user"].nunique()

In [12]:
spotted_percentage

spotted_users
Alan      0.75
Alex      0.25
Mason     0.25
Natahn    0.25
Name: from_user, dtype: float64

In [13]:
cuttoff = 0.5
flagged_users = spotted_percentage[spotted_percentage > cuttoff]

In [14]:
blacklist_url = "http://127.0.0.1:8000/locationConsensus/blacklist/"

In [15]:
def update_blacklist(users):
    for user in users:
        toSend  = {'userID': user}
        out = requests.post(blacklist_url, data = toSend)

In [16]:
update_blacklist(flagged_users.index)

In [17]:
for user in flagged_users.index:
    print(user)

Alan


In [4]:
file = open('test_data.csv')

line = file.readline()
# read in the csv (extra steps due to list as a column)
splits = line.split(',')
from_user = splits[0]
time = datetime.strptime(splits[-1].strip(), '%Y-%m-%d %H:%M:%S')

In [5]:
time

datetime.datetime(2023, 2, 9, 1, 9, 27)

In [6]:
time.astimezone(timezone.utc)

datetime.datetime(2023, 2, 9, 9, 9, 27, tzinfo=datetime.timezone.utc)

In [7]:
now = datetime.now(timezone.utc)
now

datetime.datetime(2023, 2, 16, 2, 44, 28, 543142, tzinfo=datetime.timezone.utc)

In [8]:
now.astimezone(timezone.utc)

datetime.datetime(2023, 2, 16, 2, 44, 28, 543142, tzinfo=datetime.timezone.utc)

In [9]:
trust_alg(data, time)

['alan']